In [1]:
from arithmetic_gf import gf_pow
from polynomial_gf import gf_poly_mul

def rs_generator_poly(nsym):
    '''Generate an irreducible generator polynomial (necessary to encode a message into Reed-Solomon)'''
    g = [1]
    for i in range(0, nsym):
        g = gf_poly_mul(g, [1, gf_pow(2, i)])
    return g

In [6]:
rs_generator_poly(4)

[1, 15, 54, 120, 64]